In [10]:
!pip install folium

  Using cached folium-0.11.0-py2.py3-none-any.whl (93 kB)


In [21]:
t = dict()

In [22]:
t.setdefault('name', []).append('주유소1')

In [23]:
t.setdefault('name', []).append('주유소2')

In [24]:
t.setdefault('la', []).append(35)
t.setdefault('la', []).append(36)

In [25]:
t

{'name': ['주유소1', '주유소2'], 'la': [35, 36]}

In [26]:
pd.DataFrame(t)

,name,la
0,주유소1,35
1,주유소2,36


In [11]:
import folium
from folium.plugins import MarkerCluster
import pandas as pd
import numpy as np
import os
#import seaborn as sns
#import matplotlib.pyplot as plt

In [5]:
os.path

<module 'posixpath' from '/opt/conda/lib/python3.7/posixpath.py'>

### df1과 df2로 나눈 이유는 각각 다른 마크로 찍기 위해

In [12]:
df1 = {'name': ['강릉IC주유소', '백마주유소', '충원대로주유소'], 'lo': [128.881406, 129.121687, 127.919048],
                 'la': [37.748078, 37.492835, 37.015872]}
df2 = {'name': ['강릉현대주유소', '서원경주유소', '삼천리주유소'], 'lo': [128.914091, 127.457240, 127.219048],
                 'la': [37.745318, 36.643683, 35.815872]}
df1, df2 = pd.DataFrame(df1), pd.DataFrame(df2)

In [13]:
print(df1.shape, df2.shape)
df1

(3, 3) (3, 3)


,name,lo,la
0,강릉IC주유소,128.881406,37.748078
1,백마주유소,129.121687,37.492835
2,충원대로주유소,127.919048,37.015872


In [31]:
a = list()
a.append(df1)
a.append(df2)
a.append(df3)
a

[      name          lo         la
 0  강릉IC주유소  128.881406  37.748078
 1    백마주유소  129.121687  37.492835
 2  충원대로주유소  127.919048  37.015872,
       name          lo         la
 0  강릉현대주유소  128.914091  37.745318
 1   서원경주유소  127.457240  36.643683
 2   삼천리주유소  127.219048  35.815872,
       name          lo         la
 0  강릉IC주유소  128.881406  37.748078
 1    백마주유소  129.121687  37.492835
 2  충원대로주유소  127.919048  37.015872
 0  강릉현대주유소  128.914091  37.745318
 1   서원경주유소  127.457240  36.643683
 2   삼천리주유소  127.219048  35.815872]

In [32]:
pd.concat(a)

,name,lo,la
0,강릉IC주유소,128.881406,37.748078
1,백마주유소,129.121687,37.492835
2,충원대로주유소,127.919048,37.015872
0,강릉현대주유소,128.914091,37.745318
1,서원경주유소,127.457240,36.643683
2,삼천리주유소,127.219048,35.815872
0,강릉IC주유소,128.881406,37.748078
1,백마주유소,129.121687,37.492835
2,충원대로주유소,127.919048,37.015872
0,강릉현대주유소,128.914091,37.745318


In [14]:
df3 = pd.concat([df1, df2])
df3

,name,lo,la
0,강릉IC주유소,128.881406,37.748078
1,백마주유소,129.121687,37.492835
2,충원대로주유소,127.919048,37.015872
0,강릉현대주유소,128.914091,37.745318
1,서원경주유소,127.457240,36.643683
2,삼천리주유소,127.219048,35.815872


### 일정 거리 안에 있는 주유소 출력

In [15]:
import numpy as np # 넘파이 추가되어야 함

# 거리계산 함수인데 SQRT(가로거리 제곱 + 세로거리 제곱) 로 구하는 일반적인 유클리드(?) 거리 방식
def cal_distance(a, b):
    distance = np.sqrt(np.sum((a-b)**2)) # 이거 계산할 때, a와 b는 list가 아니라 np.array 형태여야 연산 가능
    return distance
    
def check_distance(df, name, standard=3):
    
    # 목록에 없는 이름이라면 밸류에러 발생
    if name not in list(df['name']):
        raise ValueError ('그런 거 없음')
    
    data = np.array(df) # df를 array 형태로
    lo = np.array(df['lo'])
    la = np.array(df['la'])
    idx_a = list(df['name']).index(name) # name으로 전달 받은 주유소 이름의 index
    loc_a = np.array([lo[idx_a], la[idx_a]]) # lo, la에서 해당 index의 좌표값을 np.array 형태로 추출
    
    # for loop가 돌아갈 index list를 구성
    list_iter = list(range(len(data)))
    list_iter.remove(idx_a) # 본인 index를 제외한 전체 index
    
    # 스탠다드 충족하는 데이터의 index를 집어넣을 리스트
    result_idx = list()
    
    for i in list_iter:
        loc_b = np.array([lo[i], la[i]]) # 특정 index의 좌표값 추출
        if cal_distance(loc_a, loc_b) < standard: # 거리계산한 것이 기준값보다 작다면 그 index를 리스트에 추가
            result_idx.append(i)
            
    return data[result_idx] # data 중에서 해당 index에 해당하는 row만 추출

In [17]:
check_distance(df3, '서원경주유소', 1)

array([['충원대로주유소', 127.919048, 37.015872],
       ['삼천리주유소', 127.219048, 35.815872]], dtype=object)

In [101]:
t = list()
for i in range(len(df3)):
    t.append(np.array((np.array(df3['lo'])[i], np.array(df3['la'])[i])))

In [110]:
print(cal_distance(t[0], t[1]))
print(cal_distance(t[0], t[2]))
print(cal_distance(t[0], t[3]))
print(cal_distance(t[0], t[4]))
print(cal_distance(t[0], t[5]))

0.3505480680448902
1.2092388294294834
0.03280132352514131
1.8022034046080901
2.5488927295200154


In [18]:
input_1 = ['백마주유소', '서원경주유소']

In [19]:
def check_group(df, names, standard=3):
    final = dict()
    
    for name in names:
        results = check_distance(df, name, standard)
        # 예를 들어 'name', 'lo', 'la' 일 때
        final.setdefault('name', result)
    
    return final

In [20]:
t = check_group(df3, input_1, 2)
t

{'백마주유소': array([['강릉IC주유소', 128.881406, 37.748078],
        ['충원대로주유소', 127.919048, 37.015872],
        ['강릉현대주유소', 128.914091, 37.745318],
        ['서원경주유소', 127.45724, 36.643683]], dtype=object),
 '서원경주유소': array([['강릉IC주유소', 128.881406, 37.748078],
        ['백마주유소', 129.121687, 37.492835],
        ['충원대로주유소', 127.919048, 37.015872],
        ['강릉현대주유소', 128.914091, 37.745318],
        ['삼천리주유소', 127.219048, 35.815872]], dtype=object)}

In [62]:
def one_list(dict1):
    final = list()
    for key in dict1.keys():
        for station in dict1[key]:
            final.append(station[0])
    return set(final) # 중복제거하기 싫으면 set()를 빼주면 돼

In [64]:
one_list(t)

{'강릉IC주유소', '강릉현대주유소', '백마주유소', '삼천리주유소', '서원경주유소', '충원대로주유소'}

In [51]:
final = list()

for key in t.keys():
    for station in t[key]:
        result_1.append(tuple(station))

print(result_1)

[('강릉IC주유소', 128.881406, 37.748078), ('강릉현대주유소', 128.914091, 37.745318), ('강릉IC주유소', 128.881406, 37.748078), ('강릉현대주유소', 128.914091, 37.745318)]


In [46]:
t1 = [[1,2], [3,4], [1,2]]

In [50]:
temp = list()
for i in t1:
    i = tuple(i)
    temp.append(i)
set(temp)

{(1, 2), (3, 4)}

In [12]:
icon = folium.features.CustomIcon('./gasstation.png',icon_size=(20, 20))

In [27]:
m = folium.Map(location = [36.5053542, 127.70434191], zoom_start=8)

# for i in range(len(df1.name)):
#     folium.CircleMarker(location= [df1.la[i], df1.lo[i]], popup= df1.name[i], icon= folium.Icon(color='lightgreen')).add_to(m)

for i in range(len(df1.name)):
    folium.CircleMarker(location= [df1.la[i], df1.lo[i]], 
                        popup= df1.name[i], color='darkblue', fill_color='darkblue').add_to(m)
    
for i in range(len(df2.name)):
    folium.CircleMarker(location= [df2.la[i], df2.lo[i]], 
                        popup= df2.name[i], color='darkgreen', fill_color='darkgreen').add_to(m)

## to do

각 hdo 직영 주유소들 기준으로 반경 1km 내의 경쟁사 주유소들과의 게시가 차이를 주차별로 계산

    1. 주유소A를 받아서 반경 1km 내의 경쟁 주유소들의 목록을 뽑아내기
        input args: name, distance
        output: [names] which are different brands
        
    2. 특정 시점에 해당 주유소들의 게시가격 평균 구하기
        input args: date, [names]
        output: (mean(G), mean(D))
        
    3. 주유소A와 반경, 특정 시점을 기준으로 주변 경쟁사들의 평균가격과의 차이를 dict로 뽑기
        input args: name, distance, date
        output: {names: [], date: [], G_diff: [], D_diff: [], lat: [], lon: []}

In [14]:
m.save('temp.html')

In [13]:
url_soil = "https://icons8.com/icons/set/gas-station"

In [71]:
m = folium.Map(location = [36.5053542, 127.7043419], zoom_start=8)

from folium.plugins import MarkerCluster
marker_cluster = MarkerCluster().add_to(m)

for i in range(len(df.name)):
    folium.Marker(location= [df.la[i], df.lo[i]], 
                  popup= list(df.name)[i], icon= folium.Icon(color='blue', icon='ok')
                 ).add_to(marker_cluster)

In [95]:
m.save('temp.html')

In [50]:
data.T.shape

(30, 2)

In [44]:
m = folium.Map([45, 3], zoom_start=4)
MarkerCluster(data.T, popups=list(range(30))).add_to(m)

In [28]:
locations = np.array([df.la, df.lo])
locations = locations.T
locations

array([[ 37.748078, 128.881406],
       [ 37.492835, 129.121687],
       [ 37.015872, 127.919048]])

In [24]:
names = list(df.name)
names

['강릉IC주유소', '백마주유소', '충원대로주유소']

In [12]:
data = np.array([
np.random.uniform(low=35, high=60, size=30), # Random latitudes in Europe
np.random.uniform(low=-12, high=30, size=30), # Random longitudes in Europe.
])

In [18]:
type(np.random.uniform(low=-12, high=30, size=30))

numpy.ndarray

In [13]:
data.T

array([[ 4.74540576e+01,  1.12807240e+01],
       [ 3.50661151e+01,  2.88691915e+01],
       [ 4.71515916e+01,  2.52088633e+01],
       [ 4.24218244e+01, -1.17766892e+01],
       [ 4.65240633e+01,  2.24771247e+01],
       [ 4.53712740e+01,  6.28943368e+00],
       [ 4.63168865e+01, -5.30944374e-02],
       [ 5.17800244e+01, -9.75983269e+00],
       [ 4.96597794e+01,  9.30037959e+00],
       [ 5.73365321e+01,  1.26731288e+00],
       [ 5.07650766e+01, -6.04274242e+00],
       [ 4.63847936e+01, -8.73240220e+00],
       [ 4.04589994e+01,  1.39835366e+01],
       [ 4.98897627e+01,  1.39361004e+01],
       [ 5.25517839e+01, -7.48281283e+00],
       [ 4.29884346e+01, -7.01762579e+00],
       [ 5.76001771e+01, -5.42685359e+00],
       [ 4.48480781e+01,  1.93521637e+01],
       [ 4.18211978e+01,  1.59172190e+01],
       [ 5.45629561e+01, -5.32140498e+00],
       [ 5.04860695e+01, -5.53191440e+00],
       [ 4.37637905e+01,  1.85811200e+00],
       [ 4.22235066e+01,  7.41371685e+00],
       [ 5.

In [41]:
m = folium.Map([45, 3], zoom_start=4)
MarkerCluster(data.T, popups=list(range(30))).add_to(m)

In [42]:
m.save('temp.html')

In [37]:
locations.shape

(3, 2)

In [39]:
m = folium.Map(location = [36.5053542, 127.7043419], zoom_start=8)
from folium.plugins import MarkerCluster
marker_cluster = MarkerCluster().add_to(m)
folium.Marker(locations, popup=names, icon=folium.Icon(color='red', icon='ok')).add_to(marker_cluster)

ValueError: Expected two (lat, lon) values for location, instead got: [[37.748078, 128.881406], [37.492835, 129.121687], [37.015872, 127.919048]].

In [77]:
m = folium.Map(location = [36.5053542, 127.7043419], zoom_start=8)

from folium.plugins import MarkerCluster
marker_cluster = MarkerCluster().add_to(m)

for i in range(len(df.name)):
    folium.Marker(location= [df.la[i], df.lo[i]], 
                  popup= df.name[i], icon= folium.Icon(color='blue', icon='ok')
                 ).add_to(marker_cluster)


In [19]:
print(m)

In [78]:
m.save('temp.html')

In [4]:
df.lo

0    128.881406
1    129.121687
2    127.919048
Name: lo, dtype: float64